# Modules To Import

In [199]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Crawler Class
> #### Note On Season Identifiers
> Season years are represented in th from of "Season year-year+1" (e.g. Season 2021-2022). To have less data complexity and more efficieny I choose "year+1" from each season in the form of "year-year+1" as a key for that specific season because the website [basketball-reference.com](www.basketball-reference.com) is also doing the same.
> 
>  **TL;DR**: The key "2024" points to the 2023-24 season.  

In [140]:
class Crawler:
    def get_mvps(self, mvp_page):
        mvp_soup = BeautifulSoup(mvp_page.content, "html.parser")
        mvp_soup = mvp_soup.find('table', id="mvp_NBA").tbody.find_all("tr")
        mvp_df = pd.DataFrame(columns=["player_name", "player_id", "team", "year"])

        # Scraps Required Elements from each row of the table
        for tr in mvp_soup:
            mvp_df.loc[len(mvp_df)] = [      
                tr.find(attrs={"data-stat":"player"}).a.string,
                tr.find(attrs={"data-stat":"player"}).a.get("href")[9:-5],
                tr.find(attrs={"data-stat":"team_id"}).a.get("href")[1:-5].split("/")[1],
                tr.find(attrs={"data-stat":"team_id"}).a.get("href")[1:-5].split("/")[-1]
            ]
        return mvp_df

    def get_player_data(self, player_series: pd.Series, player_page):
        player_series = player_series.copy()
#        player_page = requests.get("https://www.basketball-reference.com/players/" + player_series["player_id"] + ".html")
        player_soup = BeautifulSoup(player_page.content, "html.parser")
        return player_soup
        
        

In [24]:
mvp_page = requests.get("https://www.basketball-reference.com/awards/mvp.html")

In [141]:
c = Crawler()
mvp = c.get_mvps(mvp_page)
player_page = requests.get("https://www.basketball-reference.com/players/" + mvp.iloc[0]["player_id"] + ".html")

In [236]:
jokic = c.get_player_data(mvp.iloc[0], player_page)
# experience
print(int(jokic.find("strong", string=re.compile(".*Experience:.*")).next_sibling.strip().split()[0]))
print(jokic.find("strong", string=re.compile(".*Shoots:.*")).next_sibling.strip())
print(jokic.find("span", string=re.compile(".*lb")).next_sibling.strip()[1:-1].split(",\xa0"))
mvp

9
Right
['211cm', '128kg']


,player_name,player_id,team,year
0,Nikola Jokić,j/jokicni01,DEN,2024
1,Joel Embiid,e/embiijo01,PHI,2023
2,Nikola Jokić,j/jokicni01,DEN,2022
3,Nikola Jokić,j/jokicni01,DEN,2021
4,Giannis Antetokounmpo,a/antetgi01,MIL,2020
...,...,...,...,...
64,Wilt Chamberlain,c/chambwi01,PHW,1960
65,Bob Pettit,p/pettibo01,STL,1959
66,Bill Russell,r/russebi01,BOS,1958
67,Bob Cousy,c/cousybo01,BOS,1957
